<a href="https://colab.research.google.com/github/Confirmation-Bias-Analyser/Data-Collection/blob/main/Data_Processing_of_PHEME_Dataset_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install anytree

# Import essential libraries and declare functions

In [ ]:
# from transformers import BertTokenizer, TFBertForSequenceClassification, InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
from google.colab import files
from anytree import Node, RenderTree, search
import random

# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
import json

In [ ]:
import re
import urllib
from urllib.parse import urlparse

def createTweetsTree(dictionary, tree_root):
    for key, item in dictionary.items():
        child = Node(key, parent=tree_root)

        if len(dictionary[key]) != 0:      
            createTweetsTree(dictionary[key], child)

        else:
            continue

def processTweetsToDict(conversation_data):
    conversation_dict = {'id':[], 'timestamp':[], 'reply_to':[], 'comment':[]}

    for i in conversation_data:
        print('User ID:', i['id'], 
              'Time:', i['user']['created_at'])
        print('In reply to:', i['in_reply_to_status_id'])
        print(i['text'], '\n')

        conversation_dict['id'].append(i['id'])
        conversation_dict['timestamp'].append(i['user']['created_at'])
        conversation_dict['reply_to'].append(i['in_reply_to_status_id'])
        conversation_dict['comment'].append(i['text'])

    return conversation_dict

def getLinks(string):
    urls = re.findall("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", string)
    links = ''

    for url in urls:
        try:
            opener = urllib.request.build_opener()
            request = urllib.request.Request(url)
            response = opener.open(request)
            actual_url = response.geturl()
                      
            if '](' in actual_url:
                actual_url = actual_url.split('](')[0]
          
            links += actual_url + ';'
            
            
        except:
            if '](' in url:
                url = url.split('](')[0]
          
            links += url + ';'

    return links

def getURLfromList(url):
    if ';' in url:
        url = url.split(';')[:-1]
        result = []
    
        for i in url:
            result.append(urlparse(i).hostname)
        
        return result

    else:
        return ''   

# Extract PHEME dataset and save to directory

In [ ]:
URL = "https://figshare.com/ndownloader/files/11767817/PHEME_veracity.tar.bz2"

dataset = tf.keras.utils.get_file(fname="PHEME_veracity.tar.bz2", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

In [ ]:
!tar xzf '/content/PHEME_veracity.tar.bz2.tar.gz'

In [ ]:
threads = []
data = ['non-rumours', 'rumours']

for i in os.listdir('/content/all-rnr-annotated-threads'):
  if i[0] != '.':
    threads.append(i)

## Select random sample directory

In [ ]:
selected_thread = random.choice(threads)
selected_type = random.choice(data)

path = '/content/all-rnr-annotated-threads/' + selected_thread + '/' + selected_type
count = 0
allTweets = []

print(selected_thread, selected_type)

for i in os.listdir(path):
  if i[0] != '.':
    reactions_path = path + '/' + i + '/structure.json'
    
    with open(reactions_path, 'r') as k:
      contents = json.loads(k.read())

      if len(contents[i]) > 1:
        allTweets.append(i)

    count += 1

gurlitt-all-rnr-threads rumours


## Select random tweet and read the tweet

In [ ]:
wanted_id = random.choice(allTweets)
print(wanted_id, '\n')

with open(path + '/' + wanted_id + '/structure.json', 'r') as k:
  contents = json.loads(k.read())
  
root = Node(wanted_id)
createTweetsTree(contents[wanted_id], root)

tweet_path = path + '/' + wanted_id
allComments = []

with open(tweet_path + '/source-tweets/' + wanted_id + '.json', 'r') as j:
  master_tweet= json.loads(j.read())

print(master_tweet['text'], '\n')

for j in os.listdir(tweet_path + '/reactions/'):
  if j[0] != '.':
    
    with open(tweet_path + '/reactions/' + j, 'r') as k:
      contents = json.loads(k.read())
      allComments.append(contents)

536826643122503680 

Swiss museum to accept hundreds of works of art bequeathed by German Nazi-era art hoarder Cornelius Gurlitt http://t.co/7DHYBAL0QK 



## Get random tweet
Visualise tree of comments and convert comments to dataframe

In [ ]:
for pre, fill, node in RenderTree(root):
  print("%s%s" % (pre, node.name))

536826643122503680
├── 536827740167864320
├── 536827884116385792
├── 536828004983652352
│   └── 536832872162680833
│       └── 536834142361886720
├── 536828098755715074
├── 536828276128624640
├── 536828855403945984
├── 536829095892770816
├── 536829499686793216
├── 536829987157209088
├── 536830791024922624
├── 536832724787400704
├── 536835189260177408
├── 536836102611828736
└── 536837278761775104


In [ ]:
result = processTweetsToDict(allComments)

User ID: 536835189260177408 Time: Tue Jun 16 09:03:50 +0000 2009
In reply to: 536826643122503680
@bbcbreaking At least it'll deflect attention from one of their own most psychotic sons, #SeppBlatter. #fifa #nazi #swiss 

User ID: 536829499686793216 Time: Sat Nov 08 09:11:01 +0000 2008
In reply to: 536826643122503680
@BBCBreaking It's nice 2 see the Swiss are benefitting once more from the Nazis as this rich banking country needs all the help they can get 

User ID: 536829095892770816 Time: Sun Oct 30 09:34:22 +0000 2011
In reply to: 536826643122503680
@BBCBreaking @BBCWorld not the 1st time Swiss have taken Nazi loot 

User ID: 536827884116385792 Time: Mon Mar 24 18:29:45 +0000 2014
In reply to: 536826643122503680
@BBCBreaking 
Instant Date Prank 

http://t.co/DhERLF9ytr

.

#MTVStars 
#RejectedSNLCharacters  .40 

User ID: 536834142361886720 Time: Mon Sep 01 20:02:45 +0000 2014
In reply to: 536832872162680833
@TheExposMoose @BBCBreaking Yes unfortunately the case, maybe when its hung 

In [ ]:
df = pd.DataFrame.from_dict(result)
df['id'] = df['id'].astype(str)
df['reply_to'] = df['reply_to'].astype(str)
df['url'] = df['comment'].apply(lambda x: getLinks(x))
df['link_title'] = df['url'].apply(lambda x: getURLfromList(x))

df.head()

,id,timestamp,reply_to,comment,url,link_title
0,536835189260177408,Tue Jun 16 09:03:50 +0000 2009,536826643122503680,@bbcbreaking At least it'll deflect attention ...,,
1,536829499686793216,Sat Nov 08 09:11:01 +0000 2008,536826643122503680,@BBCBreaking It's nice 2 see the Swiss are ben...,,
2,536829095892770816,Sun Oct 30 09:34:22 +0000 2011,536826643122503680,@BBCBreaking @BBCWorld not the 1st time Swiss ...,,
3,536827884116385792,Mon Mar 24 18:29:45 +0000 2014,536826643122503680,@BBCBreaking \nInstant Date Prank \n\nhttp://t...,http://t.co/DhERLF9ytr;,[t.co]
4,536834142361886720,Mon Sep 01 20:02:45 +0000 2014,536832872162680833,@TheExposMoose @BBCBreaking Yes unfortunately ...,,
